In [1]:
!pip install bs4

In [2]:
import requests
import re
from bs4 import BeautifulSoup

In [3]:
def open_profile_data(server,name,tagline):
    leagueofgraphs_url = f"https://www.leagueofgraphs.com/summoner/{server}/{name}-{tagline}"

    try:
        response = requests.get(leagueofgraphs_url, headers = {'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        return soup

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

In [4]:
name = "lurkz"
tagline = "h3art"
server = "euw"
leagueofgraphs = open_profile_data(server,name,tagline)

## Get rank of user

In [5]:
leagueofgraphs


<!DOCTYPE html>

<html lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<link href="https://fonts.googleapis.com/css?family=Open+Sans:300,400,500,700" media="none" onload="this.media='all'" rel="stylesheet" type="text/css"/> <link href="//lolg-cdn.porofessor.gg/style.css?v=1713798443" rel="stylesheet" type="text/css">
<link href="//lolg-cdn.porofessor.gg/style.sprite.css?v=fcd4db6c7b68f9a6715ed72976823a75" rel="stylesheet" type="text/css"/>
<script src="//lolg-cdn.porofessor.gg/jquery.js?v=3"></script>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="en" http-equiv="content-language"/>
<meta content="GrandMaster - Wins: 238 (56.8%) (#2,989) / Talon: Wins: 59.1% - Played: 340 (#173) / Smolder: Wins: 60.0% - Played: 15 / Hwei: Wins: 57.1% - Played: 7" name="description"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="origin" name="referrer"/>
<meta con

### Find league tier and LP

In [6]:
rank_element = leagueofgraphs.find(class_='leagueTier')
lp_element = leagueofgraphs.find(class_='leaguePoints')

rank_helper = rank_element.text.strip()
rank = re.sub(r'\s+', ' ', rank_helper) 

lp_helper = lp_element.text.strip()
lp = int(re.findall(r'\d+', lp_helper)[0])

print(f"{name}#{tagline} is ranked as: {rank}, their LP is {lp}")


lurkz#h3art is ranked as: GrandMaster 835 LP, their LP is 835


### Find the level

In [31]:
level_element = leagueofgraphs.find(class_='bannerSubtitle')

level_helper = level_element.text.strip()
level_helper_1 = re.sub(r'-.+', '', level_helper)
level_helper_2 = re.findall(r'\d+', level_helper_1)
print(level_helper_2)



['1121']


### Find wins and losses number

In [7]:
wins_element = leagueofgraphs.find(class_='winsNumber')
losses_element = leagueofgraphs.find(class_='lossesNumber')

wins = int(wins_element.text.strip())
losses = int(losses_element.text.strip())
winrate = round((wins/(wins+losses))*100, 3)
print(f"{name}#{tagline} has: {losses} losses and {wins} wins, their winrate is {winrate}")


lurkz#h3art has: 165 losses and 213 wins, their winrate is 56.349


### Get rankings

In [8]:
rank_element = leagueofgraphs.find(class_='rank')
global_rank_element = rank_element.find(class_='highlight')
regional_rank_element = rank_element.find(class_='regionalRank')
top_rank_percentage_element = rank_element.find(class_='topRankPercentage')

top_rank_percentage_helper = str(top_rank_percentage_element.string)
global_rank_helper = str(global_rank_element.string)
regional_rank_helper = re.findall(r'\d+(?:,\d+)?', regional_rank_element.string)[0]

top_rank_percentage_number = float(re.findall(r'\d+(?:.\d+)?', top_rank_percentage_helper)[0])
global_rank_number = re.sub(r',','', str(global_rank_helper))
regional_rank_number = re.sub(r',','', regional_rank_helper)

print(f"{name}#{tagline} is: Rank {global_rank_number} globally, making them be in the top {top_rank_percentage_number}%, their rank on {server} is {regional_rank_number} ")

lurkz#h3art is: Rank 2989 globally, making them be in the top 0.021%, their rank on euw is 360 


### Get all played champion

In [9]:
champions_container = leagueofgraphs.find_all(class_='tabs-content')[1] #gives back the champions content box

# tr_elements = champions_container.find_all_next(class_="content")[0].find_next('table').find_all_next('tr')[1:11] #gives back the top 10 all queue champions of account
tr_elements = champions_container.find_all_next(class_="content")[1].find_next('table').find_all_next('tr')[1:11] #gives back the top 10 soloqueue champions of account
# tr_elements = champions_container.find_all_next(class_="content")[2].find_next('table').find_all_next('tr')[1:11] #gives back the top 10 flex champions of account
# tr_elements = champions_container.find_all_next(class_="content")[4].find_next('table').find_all_next('tr')[1:11] #gives back the top 10 aram champions of account

champions_dictionary = {}

i = 0
for el in tr_elements:
    champion_dict = {}
    champion = el.find('td').find_next('div').find_next(class_='txt')

    champion_dict['name'] = champion.find(class_='name').string

    if (champion.find(class_='top-rank') == None):
        champion_dict['top_rank'] = None
        champion_dict['regional_rank'] = None
    else:
        champion_dict['top_rank'] = champion.find(class_='top-rank').string
        champion_dict['regional_rank'] = champion.find(class_='regionalRank').string
    
    champion_dict['kills'] = champion.find(class_='kills').string
    champion_dict['deaths'] = champion.find(class_='deaths').string
    champion_dict['assists'] = champion.find(class_='assists').string

    champions_dictionary[i] = champion_dict
    i += 1
print(champions_dictionary)


{0: {'name': 'Talon', 'top_rank': '173', 'regional_rank': '42', 'kills': '10.2', 'deaths': '5.9', 'assists': '5.3'}, 1: {'name': 'Naafiri', 'top_rank': None, 'regional_rank': None, 'kills': '6.4', 'deaths': '4.8', 'assists': '4.1'}, 2: {'name': 'Smolder', 'top_rank': None, 'regional_rank': None, 'kills': '8.0', 'deaths': '5.9', 'assists': '6.1'}, 3: {'name': 'Vayne', 'top_rank': None, 'regional_rank': None, 'kills': '11.1', 'deaths': '5.6', 'assists': '6.0'}, 4: {'name': 'Vex', 'top_rank': None, 'regional_rank': None, 'kills': '6.0', 'deaths': '5.7', 'assists': '6.3'}, 5: {'name': 'Katarina', 'top_rank': None, 'regional_rank': None, 'kills': '9.0', 'deaths': '8.8', 'assists': '4.3'}, 6: {'name': "Kai'Sa", 'top_rank': None, 'regional_rank': None, 'kills': '6.5', 'deaths': '4.8', 'assists': '6.5'}, 7: {'name': 'Azir', 'top_rank': None, 'regional_rank': None, 'kills': '3.8', 'deaths': '6.3', 'assists': '6.3'}, 8: {'name': 'Ekko', 'top_rank': None, 'regional_rank': None, 'kills': '7.3', 'd

In [10]:
# rank variable can be: iron, bronze, silver, gold, platinum, emerald, diamond, master
# note that when it's platinum there's no additional get request needed

def open_champion_data(rank):
    if (rank == 'platinum' or rank == '' or rank == None):
        leagueofgraphs_url = f"https://www.leagueofgraphs.com/champions/builds/sr-ranked/by-champion-name"
    else: leagueofgraphs_url = f"https://www.leagueofgraphs.com/champions/builds/{rank}/sr-ranked/by-champion-name"

    try:
        response = requests.get(leagueofgraphs_url, headers = {'User-Agent': 'Mozilla/5.0'})
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')
        return soup

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

In [11]:
rank = "emerald"
leagueofgraphs_champs = open_champion_data(rank)

In [12]:
leagueofgraphs_champs


<!DOCTYPE html>

<html lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<link href="https://fonts.googleapis.com/css?family=Open+Sans:300,400,500,700" media="none" onload="this.media='all'" rel="stylesheet" type="text/css"/> <link href="//lolg-cdn.porofessor.gg/style.css?v=1713798443" rel="stylesheet" type="text/css">
<link href="//lolg-cdn.porofessor.gg/style.sprite.css?v=fcd4db6c7b68f9a6715ed72976823a75" rel="stylesheet" type="text/css"/>
<script src="//lolg-cdn.porofessor.gg/jquery.js?v=3"></script>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="en" http-equiv="content-language"/>
<meta content="We track millions of LoL games played every day gathering champion stats, matchups, builds &amp; summoner rankings, as well as champion stats, popularity, winrate, teams rankings, best items and spells." name="description"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/

## Make JSON of datas for champions

In [13]:
leagueofgraphs_champs.find_all(class_='hide-for-dark')

tr_elements_champs = leagueofgraphs_champs.find(id='mainContent').find_next('div').find_next('div').find_next('div').find_next('table').find_all_next('tr')[1:180]

champions_stats = {}
tr_elements_champs

i = 0
for el in tr_elements_champs:
    if (i != 8 and i % 16 != 0 or i == 0): 
        cur_champ_data = {}
        champion = el.find('td')

        cur_champ_data['name'] = re.sub(r'\s+', '', champion.find_next(class_='name').string)

        cur_champ_data['popularity'] = round(float(champion.find_all_next('progressbar')[0]['data-value'])*100,2)
        cur_champ_data['winrate'] = round(float(champion.find_all_next('progressbar')[1]['data-value'])*100,2)
        cur_champ_data['banrate'] = round(float(champion.find_all_next('progressbar')[2]['data-value'])*100,2)

        cur_champ_data['avg.kills'] = float(champion.find_next(class_='kills').string)
        cur_champ_data['avg.deaths'] = float(champion.find_next(class_='deaths').string)
        cur_champ_data['avg.assists'] = float(champion.find_next(class_='assists').string)

        champions_stats[i] = cur_champ_data
    i += 1    

print(champions_stats)

{0: {'name': 'Aatrox', 'popularity': 11.25, 'winrate': 49.54, 'banrate': 14.61, 'avg.kills': 6.1, 'avg.deaths': 6.2, 'avg.assists': 5.3}, 1: {'name': 'Ahri', 'popularity': 13.21, 'winrate': 51.62, 'banrate': 16.5, 'avg.kills': 6.5, 'avg.deaths': 5.3, 'avg.assists': 7.8}, 2: {'name': 'Akali', 'popularity': 9.78, 'winrate': 49.26, 'banrate': 17.52, 'avg.kills': 8.6, 'avg.deaths': 5.9, 'avg.assists': 4.8}, 3: {'name': 'Akshan', 'popularity': 3.02, 'winrate': 50.7, 'banrate': 5.21, 'avg.kills': 8.5, 'avg.deaths': 6.4, 'avg.assists': 5.7}, 4: {'name': 'Alistar', 'popularity': 4.2, 'winrate': 49.94, 'banrate': 1.68, 'avg.kills': 1.7, 'avg.deaths': 6.5, 'avg.assists': 14.5}, 5: {'name': 'Amumu', 'popularity': 2.85, 'winrate': 48.8, 'banrate': 0.64, 'avg.kills': 4.3, 'avg.deaths': 6.3, 'avg.assists': 11.2}, 6: {'name': 'Anivia', 'popularity': 2.78, 'winrate': 51.84, 'banrate': 1.97, 'avg.kills': 5.8, 'avg.deaths': 4.9, 'avg.assists': 7.9}, 7: {'name': 'Annie', 'popularity': 2.75, 'winrate': 51